In [1]:
import sys
import os

project_root = os.path.abspath("..")
src_path = os.path.join(project_root, "src")

if src_path not in sys.path:
    sys.path.insert(0, src_path)

In [2]:
import numpy as np

# === Shifts ===
def shift_right(mat, steps=1):
    return np.roll(mat, shift=steps, axis=1)

def shift_left(mat, steps=1):
    return np.roll(mat, shift=-steps, axis=1)

def shift_down(mat, steps=1):
    return np.roll(mat, shift=steps, axis=0)

def shift_up(mat, steps=1):
    return np.roll(mat, shift=-steps, axis=0)

def shift_diag_down_right(mat, steps=1):
    return np.roll(np.roll(mat, shift=steps, axis=0), shift=steps, axis=1)

def shift_diag_down_left(mat, steps=1):
    return np.roll(np.roll(mat, shift=steps, axis=0), shift=-steps, axis=1)

def shift_diag_up_right(mat, steps=1):
    return np.roll(np.roll(mat, shift=-steps, axis=0), shift=steps, axis=1)

def shift_diag_up_left(mat, steps=1):
    return np.roll(np.roll(mat, shift=-steps, axis=0), shift=-steps, axis=1)

# === New: Rotations (in 90-degree steps) ===
def rotate_clockwise(mat, k=1):
    """Rotate 90 degrees clockwise k times (k=1 means 90°, k=2 means 180°, etc.)"""
    return np.rot90(mat, -k)

def rotate_counterclockwise(mat, k=1):
    """Rotate 90 degrees counterclockwise k times (k=1 means 90°, k=2 means 180°, etc.)"""
    return np.rot90(mat, k)

def invert_bits(mat):
    """Assumes binary matrix of 0s and 1s"""
    return 1 - mat

def spiral_clockwise(mat):
    """
    Shifts all elements in the matrix one step in a clockwise spiral pattern.
    Assumes a 2D square matrix.
    """
    mat = mat.copy()
    n, m = mat.shape
    assert n == m, "Spiral transform currently supports only square matrices."
    
    res = mat.copy()
    layers = (n + 1) // 2  # Number of layers in the spiral

    for layer in range(layers):
        top = layer
        bottom = n - layer - 1
        left = layer
        right = m - layer - 1

        # Save last element of the current layer to wrap around
        last = mat[top + 1][left]

        # Move left column up
        for i in range(top + 1, bottom + 1):
            res[i - 1][left] = mat[i][left]

        # Move bottom row left
        for i in range(left + 1, right + 1):
            res[bottom][i - 1] = mat[bottom][i]

        # Move right column down
        for i in range(bottom - 1, top - 1, -1):
            res[i + 1][right] = mat[i][right]

        # Move top row right
        for i in range(right - 1, left - 1, -1):
            res[top][i + 1] = mat[top][i]

        # Place saved last element
        res[top][left + 1] = last

    return res



In [3]:
transformation = invert_bits 

## Task Definition

In [4]:
# Example 1
x1 = np.array([
    [1, 1, 0, 1],
    [1, 1, 0, 1],
    [1, 1, 0, 1],
    [1, 1, 0, 1]
], dtype=np.uint8)

# Example 2
x2 = np.array([
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
], dtype=np.uint8)

# Example 3
x3 = np.array([
    [1, 0, 1, 1],
    [1, 0, 1, 1],
    [1, 0, 1, 1],
    [1, 0, 1, 1]
], dtype=np.uint8)

# # Example 4
# x4 = np.array([
#     [1, 1, 0, 0],
#     [1, 1, 0, 0],
#     [0, 0, 0, 0],
#     [0, 0, 0, 0]
# ], dtype=np.uint8)

# # Example 5
# x5 = np.array([
#     [0, 0, 1, 1],
#     [0, 0, 1, 1],
#     [0, 0, 0, 0],
#     [0, 0, 0, 0]
# ], dtype=np.uint8)

# Test
x_test = np.array([
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 1, 1],
    [0, 0, 1, 1]
], dtype=np.uint8)


train_xs = [x1, x2, x3]
train_ys = [transformation(x) for x in train_xs]

train_xs = np.stack(train_xs)
train_ys = np.stack(train_ys)

y_test = transformation(x_test)


## Abduction

In [5]:
from algorithmic_inference import AlgorithmicAbductionInduction

# Initialize the abducer
abducer = AlgorithmicAbductionInduction(num_rules=1_000_000, seed=42, boundary_mode=1)

# # Filter training data based on CTM similarity to x_test
filtered_xs, filtered_ys = abducer.filter_by_ctm_input(train_xs, train_ys, x_test, mode='absolute', threshold=2.0)

print(f"[Filtering] Retained {len(filtered_xs)} of {len(train_xs)} examples (ruled out {len(train_xs) - len(filtered_xs)})")

# Run abduction on filtered training examples
rule_matches = abducer.abduct_rules(filtered_xs, filtered_ys)


/Users/eyujis/Library/Caches/pypoetry/virtualenvs/algorithmic-arc-E897BRAD-py3.11/lib/python3.11/site-packages/pybdm/utils.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream


[Filter] x_test complexity: 26.79 (key: 0000000000110011)
[Filter] Retaining 3 / 3 training examples (mode=absolute, threshold=2.0)
[Filtering] Retained 3 of 3 examples (ruled out 0)
[Abduction] Searching for CA rules matching 3 training pairs...
[Abduction] Found 206 rules that match all training pairs.


## Ranking by Complexity via BDM

In [6]:
# Rank rules by BDM (1D complexity of 512-bit rule encoding)
ranked_rules = abducer.rank_abducted_rules_by_bdm(list(rule_matches.keys()))

# Preview top-ranked rules
for i, (rule, score) in enumerate(ranked_rules[:5]):
    print(f"#{i+1}: Rule {rule} — BDM = {score:.4f}")

[Ranking] Ranking 206 rules by BDM complexity...
  Top 1: Rule 6963596591470305002644758069122268265407427564905681393920647450365539577928429027775975748099757603289888210677119408101648403017894252246440762022717638 → BDM = 1277.20
  Top 2: Rule 6929464058022831482422511136282689249546783701341824166703737440977330906309606471264082564820479889248694322337256253378271813547786654817591594390401115 → BDM = 1285.75
  Top 3: Rule 2982083521103024055739392287248623260248763282030791005198342655596222421505648609370518491559515380937079310042843274131398217047669148777196024325410823 → BDM = 1296.80
  Top 4: Rule 8857368800680804046740289526766717613625953919700173928681565965379570079189494721364183052064756109955602275575436172510440151241247105433243112174599077 → BDM = 1308.10
  Top 5: Rule 3374620394797911009816601006786740534688079712398148032972611290867654815846266514238526626253592648162885332223873492195243616636582772101501563379531743 → BDM = 1308.43
#1: Rule 69635965914703050

## Induction

In [7]:
induced_outputs = abducer.induce_outputs_from_rules(x_test, [r for (r, _) in ranked_rules])

[Induction] Applying 206 rules to x_test...
[Induction] Generated 25988 unique output candidates from rule applications.


In [8]:
k_ctm_scores = abducer.estimate_k_ctm(induced_outputs, total_rules=len(ranked_rules), time_metric='t_min')

print("First 3 estimated conditional complexities:\n")
for i, (y_key, meta) in enumerate(list(k_ctm_scores.items())[:3]):
    matrix = meta['matrix']
    k_val = meta['k_ctm']
    num_rules = meta['num_rules']
    m_prob = meta['m_y_given_x']
    t_min = meta['t_min']
    t_mean = meta['t_mean']

    print(f"Output {i + 1}:")
    print(matrix)
    print(f"K(y'|x) = {k_val:.4f}")
    print(f"# Matching Rules = {num_rules}, m(y|x) = {m_prob:.4e}, t_min = {t_min}, t_mean = {t_mean:.2f}")
    print("-" * 40)



[Estimate] Estimating K(y|x_test) for 25988 candidates using 206 rules...
  y'#1: k_ctm = inf, m(y|x) = 0.00000, t_min = 0
  y'#2: k_ctm = 5.69, m(y|x) = 0.01942, t_min = 1
  y'#3: k_ctm = 8.69, m(y|x) = 0.00485, t_min = 2
First 3 estimated conditional complexities:

Output 1:
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 1 1]]
K(y'|x) = inf
# Matching Rules = 206, m(y|x) = 0.0000e+00, t_min = 0, t_mean = 0.00
----------------------------------------
Output 2:
[[1 1 1 1]
 [0 1 1 1]
 [1 1 0 1]
 [0 0 1 0]]
K(y'|x) = 5.6865
# Matching Rules = 4, m(y|x) = 1.9417e-02, t_min = 1, t_mean = 198.50
----------------------------------------
Output 3:
[[0 1 0 0]
 [0 0 1 0]
 [0 1 1 0]
 [1 1 0 1]]
K(y'|x) = 8.6865
# Matching Rules = 1, m(y|x) = 4.8544e-03, t_min = 2, t_mean = 2.00
----------------------------------------


In [9]:
best_hypothesis, sorted_candidates = abducer.select_inductive_hypothesis(k_ctm_scores)

print("Top 10 ranked inductive hypotheses:\n")
for i, (y_key, meta) in enumerate(sorted_candidates[:10]):
    matrix = meta['matrix']
    k_val = meta['k_ctm']
    num_rules = meta['num_rules']
    m_prob = meta['m_y_given_x']
    t_min = meta['t_min']
    t_mean = meta['t_mean']

    print(f"Rank {i + 1}:")
    print(matrix)
    print(f"K(y'|x) = {k_val:.4f}")
    print(f"# Matching Rules = {num_rules}, m(y|x) = {m_prob:.4e}, t_min = {t_min}, t_mean = {t_mean:.2f}")
    print("-" * 40)

[Selection] Selecting from 25988 candidate outputs...
[Selection] Best output selected with k_ctm = 0.00
Top 10 ranked inductive hypotheses:

Rank 1:
[[1 1 1 1]
 [1 1 1 1]
 [1 1 0 0]
 [1 1 0 0]]
K(y'|x) = 0.0000
# Matching Rules = 206, m(y|x) = 1.0000e+00, t_min = 1, t_mean = 79.89
----------------------------------------
Rank 2:
[[0 1 0 1]
 [0 0 0 1]
 [1 1 1 0]
 [0 1 0 1]]
K(y'|x) = 5.3646
# Matching Rules = 5, m(y|x) = 2.4272e-02, t_min = 1, t_mean = 152.80
----------------------------------------
Rank 3:
[[1 1 1 1]
 [0 1 1 1]
 [1 1 0 1]
 [0 0 1 0]]
K(y'|x) = 5.6865
# Matching Rules = 4, m(y|x) = 1.9417e-02, t_min = 1, t_mean = 198.50
----------------------------------------
Rank 4:
[[0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [1 1 1 0]]
K(y'|x) = 5.6865
# Matching Rules = 4, m(y|x) = 1.9417e-02, t_min = 1, t_mean = 43.50
----------------------------------------
Rank 5:
[[0 1 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 1 1]]
K(y'|x) = 5.6865
# Matching Rules = 4, m(y|x) = 1.9417e-02, t_min = 1, t_mean = 9

In [10]:
for i, (y_key, meta) in enumerate(sorted_candidates):
    if np.array_equal(meta['matrix'], y_test):
        print(f"\nGround-truth y_test found at rank {i + 1} with K(y'|x) = {meta['k_ctm']:.4f}")
        break
else:
    print("\nGround-truth y_test not found among candidates.")



Ground-truth y_test found at rank 1 with K(y'|x) = 0.0000
